In [ ]:
#https://docs.unsloth.ai/tutorials/how-to-finetune-llama-3-and-export-to-ollama
#https://www.datacamp.com/tutorial/llama3-fine-tuning-locally

In [1]:
import ollama

In [4]:
ollama.pull('llama3')

{'status': 'success'}

In [9]:
messages = [
  {
    'role': 'user',
    'content': 'What is the best way to learn anything?',
  },
]
response = ollama.chat('llama3', messages=messages)

In [11]:
response

{'model': 'llama3',
 'created_at': '2024-07-17T01:51:49.0606717Z',
 'message': {'role': 'assistant',
  'content': "There are many effective ways to learn anything, and the best approach often depends on the individual, the subject matter, and personal learning style. Here are some of the most popular and effective methods:\n\n1. **Spaced repetition**: Review material at increasingly longer intervals to help solidify it in your long-term memory.\n2. **Active recall**: Actively try to remember information rather than simply re-reading it. Quiz yourself or test your understanding with flashcards.\n3. **Chunking**: Break down complex information into smaller, manageable chunks. This helps to reduce overwhelm and improve comprehension.\n4. **Mnemonics**: Use associations, acronyms, or other memory aids to help encode information in your brain.\n5. **Practice testing**: Test yourself on the material you're trying to learn. This helps to identify areas where you need to focus your studying.\n